In [ ]:
import numpy as np
import pandas as pd

In [ ]:
puma_acs_data = pd.read_csv("data/ACS_2018.csv")
puma_lodes_data = pd.read_csv("data/wac_puma.csv")

puma_acs_data["PUMA"] = puma_acs_data["PUMA"].astype(str).str.zfill(7)
puma_acs_data = puma_acs_data.set_index("PUMA")

puma_lodes_data["puma"] = puma_lodes_data["puma"].astype(str).str.zfill(7)
puma_lodes_data = puma_lodes_data.set_index("puma")

In [ ]:
df = pd.read_csv("data/psam_p21.csv")
df

In [ ]:
df_h = pd.read_csv("data/psam_h21.csv")
df_h

In [ ]:
df_h[df_h["HUPAOC"] == 4][["SERIALNO", "HUPAOC"]]

In [ ]:
child = df_h[["HUPAOC", "SERIALNO"]].groupby("SERIALNO").first()
df = pd.merge(df, child, left_on="SERIALNO", right_index=True, how="left")
df["child"] = np.where(df["HUPAOC"] == 4, 0, 1)
df["child"]

In [ ]:
df["child"].value_counts()

In [ ]:
df["FER_CL"] = df["FER"].fillna(0)
df["FER_CL"] = np.where(df["FER_CL"] == 2, 0, df["FER_CL"])
df["FER_CL"].value_counts()

In [ ]:
rec_child = df.groupby("SERIALNO")["FER_CL"].max()

In [ ]:
df["REC_CHILD"] = df["SERIALNO"].apply(lambda x: rec_child.loc[x])

In [ ]:
df_h[["HUPAOC", "SERIALNO"]].groupby("SERIALNO").last()

In [ ]:
# df_household = pd.read_csv("./psam_h21.csv")
# df_household = df_household[["SERIALNO", "NP", "TYPE", "RNTP", "TEN", "VALP", "VEH", "GRNTP", "GRPIP", "HINCP", "MV", "OCPIP", "SMOCP", "TAXAMT", "WIF", "WKEXREL"]].set_index("SERIALNO")
# df_household

In [ ]:
# df = pd.merge(df_person, df_household, how="left", on="SERIALNO")
# df

In [ ]:
distances = pd.read_csv("data/ky_distances.csv").set_index("Unnamed: 0")
distances.index = distances.index.astype(str)
distances.columns = distances.columns.map(str)
distances

In [ ]:
df_subset = df[df["AGEP"] >= 18]

In [ ]:
df_subset["MIGPUMA"] = df_subset["MIGPUMA"].fillna(0)
df_subset["MIGSP"] = df_subset["MIGSP"].fillna(0)
df_subset["MIGPUMA1"] = df_subset["MIGSP"].astype(int).astype(str).str.zfill(2) + df_subset["MIGPUMA"].astype(int).astype(str).str.zfill(5)
df_subset["PUMA1"] = df_subset["ST"].astype(str).str.zfill(2) + df_subset["PUMA"].astype(str).str.zfill(5)
df_subset[["MIGPUMA1", "PUMA1"]]

In [ ]:
# all migrations within Kentucky to Kentucky

df_subset = df_subset[(df_subset["MIGPUMA1"] == "0000000") | (df_subset["MIGPUMA1"].isin(list(distances.index.astype(str))))]
df_subset

In [ ]:
df_subset["STAY"] = np.where(df_subset["MIGPUMA1"] == "0000000", 1, 0)

In [ ]:
df_subset = df_subset.rename(columns={"PUMA1":"CHOSEN", "MIGPUMA1": "ORIGIN"})

In [ ]:
puma_migpuma = pd.read_excel("data/puma_migpuma.xlsx", dtype="str")
puma_migpuma["PUMA"] = puma_migpuma["State"] + puma_migpuma["PUMA"]
puma_migpuma["MIGPUMA"] = puma_migpuma["State"] + puma_migpuma["MIGPUMA"]
puma_migpuma = puma_migpuma.set_index("PUMA")
puma_migpuma.head()

In [ ]:
for i, person in df_subset.iterrows():
    if person["ORIGIN"] == "0000000":
        df_subset.loc[i, "ORIGIN"] = puma_migpuma.loc[person["CHOSEN"], "MIGPUMA"]

In [ ]:
# times = pd.read_csv("ky_times.csv").set_index("Unnamed: 0")

# times.index = times.index.astype(str)
# times.columns = times.columns.map(str)

# times

In [ ]:
# creating alternatives
for i, puma in enumerate(distances.columns):
    df_subset["ALT" + str(i + 1) + "_PUMA"] = puma
    df_subset["ALT" + str(i + 1) + "_INTERNAL_MOVE"] = 0
    df_subset["ALT" + str(i + 1) + "_POP"] = puma_acs_data.loc[puma, "Total Population"]
    df_subset["ALT" + str(i + 1) + "_EMP"] = puma_lodes_data.loc[puma, "TOT_JOBS"]

In [ ]:
distance_matrix = pd.read_csv("data/puma_distance_matrix.csv", dtype={"Unnamed: 0":str}).set_index("Unnamed: 0")
time_matrix = pd.read_csv("data/puma_time_matrix.csv", dtype={"Unnamed: 0":str}).set_index("Unnamed: 0")
distance_matrix

In [ ]:
naics_key = pd.read_csv("data/puma_naics.txt", sep=" ", header=None).set_index(0)
naics_key = naics_key.T.to_dict("list")
naics_key = {key:value[0] for key, value in zip(naics_key.keys(), naics_key.values())}
naics_key

In [ ]:
df_subset["NAICS"] = df_subset["NAICSP"].map(naics_key)
df_subset["NAICS"] = df_subset["NAICS"].str.strip(".,").str[0:3]

In [ ]:
df_subset["NAICS"] = df_subset["NAICS"].replace("Une", np.NaN).replace("MIL", np.NaN)

In [ ]:
df_subset["NAICS"].value_counts()

In [ ]:
puma_lodes_data

In [ ]:
def isNaN(num):
    return num != num

In [ ]:
for i, person in df_subset.iterrows():
    print(i)
    job = person["NAICS"]
    if isNaN(job):
        continue
    for j, puma in enumerate(distances.columns):
        df_subset["ALT" + str(j + 1) + "_OWNJOB"] = puma_lodes_data.loc[puma, job]

In [ ]:
for i, puma in enumerate(distances.columns):
    for ind in ["MED", "MFG", "RET", "EDU", "ADM", "FOD", "CON", "PRF", "TRN", "SRV", "FIN", "WHL", "AGR", "PUB", "INF", "ENT", "REL", "UTL", "EXT", "MNG"]:
        df_subset["ALT" + str(i + 1) + "_" + ind] = puma_lodes_data.loc[puma, ind]

In [ ]:
# adding time and distance data for each one
for migpuma in distances.index:
    for j, puma in enumerate(distances.columns):
        num = j + 1
        key = "ALT" + str(num) + "_"
        dist = distance_matrix.loc[migpuma, puma]
        time = time_matrix.loc[migpuma, puma]
        df_subset.loc[df_subset["ORIGIN"] == migpuma, key + "TIME"] = time_matrix.loc[migpuma, puma]
        df_subset.loc[df_subset["ORIGIN"] == migpuma, key + "DIST"] = distance_matrix.loc[migpuma, puma]
        if dist == 0:
            df_subset.loc[df_subset["ORIGIN"] == migpuma, key + "INTERNAL_MOVE"] = 1

In [ ]:
for column in df_subset.columns:
    print(column)

In [ ]:
migpuma_acs_data = pd.read_csv("data/ACS_MIGPUMA_2018.csv").drop("Geo_FIPS", axis=1)
migpuma_lodes_data = pd.read_csv("data/wac_migpuma.csv")

migpuma_acs_data = migpuma_acs_data.rename({"Total Population": "TOT_POP", "Population Density (Per Sq. Mile)": "DENS", "Total Population Under 18 Years": "POP_UND_18", "Total Population 18 to 34 Years": "POP_18_34", "Total Population 35 to 64 Years": "POP_35_64", "Total Population 65 and Over": "POP_OVER_65", "Median Age": "MED_AGE", "Population 25 Years and Over Less than High School": "EDU_NOHIGH", "Population 25 Years and Over Some College": "EDU_SOMECOL", "Population 25 Years and Over Bachelors Degree": "EDU_BACH", "Population 25 Years and Over Masters Degree": "EDU_MASTERS", "Population 25 Years and Over Professional School Degree": "EDU_PROF", "Population 25 Years and Over Doctorate Degree": "EDU_PHD", "Population 3 Years and Over Enrolled in School Public School College": "ENRL_PUB_COLL", "Population 3 Years and Over Enrolled in School Private School College": "ENRL_PRI_COLL", "Gini Index": "GINI", "SE_A14022_001": "AGG_INC", "Median Household Income (In 2019 Inflation Adjusted Dollars)": "HH_MED_INC", "Occupied Housing Units": "HU_OCC", "Occupied Housing Units Owner Occupied": "HU_OCC_OWN", "Occupied Housing Units Renter Occupied": "HU_OCC_RENT", "Housing Units": "HU_TOT", "Housing Units Vacant": "HU_VAC", "Median Value": "HH_MED_VAL", "Median Gross Rent": "HH_MED_RENT", "Median Gross Rent as a Percentage of Household  Income": "HH_MED_RENT_PCT_INC", "Median Selected Monthly Owner Costs as a  Percentage of Household Income": "HH_MED_OWN_PCT_INC", "Total Population Foreign Born": "FOREIGN_BORN", "Population 25 Years and Over High School Graduate (Includes Equivalency)": "EDU_HIGH"}, axis=1)

In [ ]:
migpuma_acs_data

In [ ]:
migpuma_acs_data["UNEMP"] = migpuma_acs_data["Population 16 Years and Over in Labor Force Civilian Unemployed"] / migpuma_acs_data["Population 16 Years and Over in Labor Force Civilian"]

In [ ]:
temp = migpuma_acs_data[["MIGPUMA", "TOT_POP", "DENS", "POP_UND_18", "POP_18_34", "POP_35_64", "POP_OVER_65", "MED_AGE", "EDU_NOHIGH", "EDU_SOMECOL", "EDU_HIGH", "EDU_BACH", "EDU_MASTERS", "EDU_PROF", "EDU_PHD", "ENRL_PUB_COLL", "ENRL_PRI_COLL", "GINI", "AGG_INC", "HH_MED_INC", "HU_OCC", "HU_OCC_OWN", "HU_OCC_RENT", "HU_TOT", "HU_VAC", "HH_MED_VAL", "HH_MED_RENT", "HH_MED_RENT_PCT_INC", "HH_MED_OWN_PCT_INC", "FOREIGN_BORN", "UNEMP"]]
temp.columns = temp.columns.map(lambda x: str(x) + "_ORIG")
migpuma_lodes_data.columns = migpuma_lodes_data.columns.map(lambda x: str(x) + "_ORIG")
df_subset = pd.merge(df_subset, temp, left_on="ORIGIN", right_on="MIGPUMA_ORIG", how="left").merge(migpuma_lodes_data, left_on = "ORIGIN", right_on="MIGPUMA_ORIG", how="left")

In [ ]:
df_subset.to_csv("ky_estdata_full.csv")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# chosen_dists = []
# for i, mover in df_subset.iterrows():
#     chosen_dists.append(distance_matrix.loc[mover["ORIGIN"], mover["CHOSEN"]])


In [ ]:
# unchosen_dists = []
# for i, mover in df_subset.iterrows():
#     chosen = mover["CHOSEN"]
#     for puma in distances.columns:
#         if not chosen == puma:
#             unchosen_dists.append(distance_matrix.loc[mover["ORIGIN"], chosen])
        

In [ ]:
puma_acs_data["UNEMP"] = puma_acs_data["Population 16 Years and Over in Labor Force Civilian Unemployed"] / puma_acs_data["Population 16 Years and Over in Labor Force Civilian"]

In [ ]:
variable = "Median Household Income (In 2019 Inflation Adjusted Dollars)"
chosens = []
unchosens = []

for i, mover in df_subset.iterrows():
    chosen = mover["CHOSEN"]
    orig = mover["HH_MED_INC_ORIG"]
    chosens.append((int(puma_acs_data.loc[chosen, variable]) - int(orig)) / int(orig))
    for puma in distances.columns:
        if not chosen == puma:
            unchosens.append((int(puma_acs_data.loc[puma, variable]) - int(orig)) / int(orig))


In [ ]:
sns.set(rc = {'figure.figsize': (25, 8)})

In [ ]:
max(chosens)

In [ ]:
sns.displot(chosens, rug=True)
sns.displot(unchosens, rug=True)

In [ ]:
df_subset["ESR"].value_counts()

In [ ]:
df_subset["MIL"].value_counts()

In [ ]:
puma_acs_data["Total Population"].std()

In [ ]:
sns.kdeplot(puma_acs_data["Total Population"])